# Brewery Review NLP Workbook

The purpose of this notebook is to discover and demonstrate the NLP pipeline we will use to process brewery reviews into short phrases.

## Examples from Google Maps

* Mike Hess Brewing, North Park - Modern, family-friendly hangout featuring a tasting room, board games & brewery views.
* North Park Beer Company - Spacious brewery & taproom serving San Diego-style ales & lagers amid dapper decor.
* Rip Current Brewing North Park - This chill, wood-paneled joint with a roomy bar offers pub grub & house-brewed craft beers on tap.

## Notes

* Sentiment - Only analyze positive sentiment reviews for the descriptions. Though it might be interesting to experiement with including both positive and negative reviews.
* Opinions - Some reviews include only opinions about a place and may not be accurate or useful. 
* Named Entity Recognition - This will be interesting to process to see if the models are able to match the right entities.
* Context-free Grammar - The Google Maps are very elegant which makes me think they are using a more sophisticated model to generate the sentences than what I was planning. 
* Beer Styles - Beer Advocate has current beers and style so it will be straight-forward to scrape the most popular styles to include with the descriptions. 
* Natural Language Generation (NLG) - How to generate the sentences or phrases so that they make sense?
* Markov Chains - Simple solution might be to use [`markovify`](https://github.com/jsvine/markovify) to do the simple generation. It will be very dependant on the reviewer's writing style, but combining all into one might regularize.

In [1]:
import pandas as pd
import markovify
import spacy
from spacy import displacy
import markovify
from textblob import TextBlob

In [2]:
file = open('reviews_san-diego_half-door-brewing-co.txt')
reviews = file.readlines()
reviews[:3]

['Nice old fashion building. I love the door knobs for beer taps. The beer selection offers over ten taps. The bar was rented for the night for an association I belong to. There were a couple of hot banquet offerings. They were slightly different and pretty good flavor. The service was good for the event.\n',
 "This place feels more like a bar than a restaurant, but they do pour their own and other it in flights. Located blocks from Petco Park with an inviting porch, it definitely caters to Padres fans and out-of-towners in for a visit. I can't say I loved the beers I had there, but the bartender was helpful an courteous and Half Door was a more than inviting place to kill a happy hour.\n",
 'Old converted house in downtown with porch/ patio seating. L-shaped bar seats about 15... additional seat around. 16 beers on tap, half theirs, half local guests... nice mix. Friendly local vibe. Good service. Very tasty food.\n']

In [3]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(reviews[0])
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

Nice nice ADJ JJ amod Xxxx True False
old old ADJ JJ amod xxx True False
fashion fashion NOUN NN compound xxxx True False
building building NOUN NN ROOT xxxx True False
. . PUNCT . punct . False False
I -PRON- PRON PRP nsubj X True True
love love VERB VBP ROOT xxxx True False
the the DET DT det xxx True True
door door NOUN NN compound xxxx True False
knobs knob VERB VBZ dobj xxxx True False
for for ADP IN prep xxx True True
beer beer NOUN NN compound xxxx True False
taps tap NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False
The the DET DT det Xxx True True
beer beer NOUN NN compound xxxx True False
selection selection NOUN NN nsubj xxxx True False
offers offer VERB VBZ ROOT xxxx True False
over over ADP IN prep xxxx True True
ten ten NUM CD nummod xxx True True
taps tap NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False
The the DET DT det Xxx True True
bar bar NOUN NN nsubjpass xxx True False
was be AUX VBD auxpass xxx True True
rented rent VERB VBN ROOT xxxx Tru

In [4]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

ten 95 98 CARDINAL
the night 128 137 TIME


In [5]:
noun_chunks = list(doc.noun_chunks)
for chunk in noun_chunks:
    print(chunk.text)

print("\nSentences")
print("---------")
sentences = list(doc.sents)
for sent in sentences:
    print(sent.text)

Nice old fashion building
I
beer taps
The beer selection
ten taps
The bar
the night
an association
I
a couple
hot banquet offerings
They
slightly different and pretty good flavor
The service
the event

Sentences
---------
Nice old fashion building.
I love the door knobs for beer taps.
The beer selection offers over ten taps.
The bar was rented for the night for an association I belong to.
There were a couple of hot banquet offerings.
They were slightly different and pretty good flavor.
The service was good for the event.



In [6]:
displacy.render(doc, style="ent")

In [7]:
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style="dep")

In [8]:
# Get raw text as string.
with open("reviews_san-diego_half-door-brewing-co.txt") as f:
    text = f.read()

# Build the Markov model with ALL reviews
text_model = markovify.Text(text, state_size=2)

In [9]:
# Print five randomly-generated sentences
for _ in range(5):
    print(text_model.make_sentence(tries=100))

16 beers on tap.
Located in a heartbeat.
What a fantastic discovery on our first day in San Diego visit, I hit this place up two times.
Very laid back place with delicious food - I had their meat pie with braised short rib.
They also had several guest beers on tap, half theirs, half local guests... nice mix.


In [10]:
# Print three randomly-generated sentences of no more than 280 characters
for _ in range(5):
    print(text_model.make_short_sentence(144))

Everything I had was very nice and offered samples of their house stuff which was really good, The house IPA was nice.
Everything I had were amazing.
What a fantastic discovery on our first day in San Diego visit, I hit this place is a few guest, such as Monkey's Paw.
Half-Door should be included in any Gaslight/East Village crawl - it is a recent entry to San Diego's magnificent beer scene.
Prices are reasonable and service was good for the event.


In [25]:
# Sentiment analysis for each review
for review in reviews:
    doc = nlp(review)
    displacy.render(doc, style="ent")
    review_text = TextBlob(review)
    print(f"Polarity     : {review_text.sentiment.polarity}")
    print(f"Subjectivity : {review_text.sentiment.subjectivity}")
    print("\n-----\n")

Polarity     : 0.32
Subjectivity : 0.5449999999999999

-----



Polarity     : 0.3787037037037037
Subjectivity : 0.6490740740740741

-----



Polarity     : 0.1824074074074074
Subjectivity : 0.3259259259259259

-----



/Users/chris/opt/anaconda3/envs/brewery-review-nlp/lib/python3.9/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Polarity     : 0.6
Subjectivity : 1.0

-----



...

Polarity     : 0.26999999999999996
Subjectivity : 0.46153846153846156

-----



Polarity     : 0.21711484593837535
Subjectivity : 0.4300290885585003

-----



Polarity     : 0.4800000000000001
Subjectivity : 0.5700000000000001

-----



Polarity     : 0.3773148148148149
Subjectivity : 0.6148148148148148

-----



Polarity     : 0.35000000000000003
Subjectivity : 0.53125

-----



In [13]:
# Noun phrases
review_text.noun_phrases

WordList(['nice', 'old fashion building', 'door knobs', 'beer taps', 'beer selection', 'hot banquet offerings', 'pretty good flavor'])

In [14]:
# Sentiment and Subjectivity for review as a whole
review_text.sentiment

Sentiment(polarity=0.32, subjectivity=0.5449999999999999)

In [15]:
# Sentiment and Subjectivity for each sentence
for sentence in review_text.sentences:
    print(sentence)
    print(sentence.sentiment)
    print()

Nice old fashion building.
Sentiment(polarity=0.35, subjectivity=0.6)

I love the door knobs for beer taps.
Sentiment(polarity=0.5, subjectivity=0.6)

The beer selection offers over ten taps.
Sentiment(polarity=0.05, subjectivity=0.0)

The bar was rented for the night for an association I belong to.
Sentiment(polarity=0.0, subjectivity=0.0)

There were a couple of hot banquet offerings.
Sentiment(polarity=0.25, subjectivity=0.8500000000000001)

They were slightly different and pretty good flavor.
Sentiment(polarity=0.31666666666666665, subjectivity=0.7333333333333334)

The service was good for the event.
Sentiment(polarity=0.7, subjectivity=0.6000000000000001)

